In [159]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import time

import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd

import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element

import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import statistics as s

import haversine as hs
load_dotenv()

True

# Filtering and extracting the data

In [29]:
client = MongoClient("localhost:27017")
db = client["ironhack"]
c = db.get_collection("companies")

In [30]:
# c.distinct("category_code")

In [31]:
def filtering_companies():

    condition1 = {"total_money_raised" : {"$regex": "\d{2,}\.?\d?M|\d{1,}\.?\d?B"}}
    condition2 = {"tag_list": {"$regex": ".*design.|.tech.|.software.|.fashion."}}
    condition3 = {"category_code": "web"}
    condition4 = {"offices": {"$ne": []}}

    projection = {"_id": 0, "name":1, "offices.city":1, "offices.latitude":1, "offices.longitude": 1, "address1":1, "total_money_raised": 1}

    query = {"$and": [condition1, condition4, {"$or": [condition2, condition3]}]}


    filtered_companies = list(c.find(query, projection))
    df = pd.DataFrame(filtered_companies)
    
    return df


In [32]:
df = filtering_companies()
df.sample(1)

,name,total_money_raised,offices
287,Swype,$13.9M,"[{'city': 'Seattle', 'latitude': 47.6495905, '..."


In [33]:
def cleaning_companies_df(df):

    df = df.explode("offices")
    df.reset_index(drop=True)
    
    cities = []
    latitudes = []
    longitudes = []
    addresses = []
    money = []

    for index, row in df.iterrows():

        try:
            cities.append(row["offices"]["city"])
            latitudes.append(row["offices"]["latitude"])
            longitudes.append(row["offices"]["longitude"])

        except IndexError:
            cities.append(None)
            latitudes.append(None)
            longitudes.append(None)


    df["city"] = cities
    df["latitude"] = latitudes
    df["longitude"] = longitudes
    
    df = df.drop("offices", axis=1)
    
    df["total_money_raised"] = df["total_money_raised"].replace('M|\$|€|C|£', "", regex = True).replace('B', "000", regex = True)
    df["total_money_raised"] = pd.to_numeric(df["total_money_raised"])

    df.to_csv("filtered_companies.csv", index=False)
    
    return df


In [34]:
df2 = cleaning_companies_df(df)

In [35]:
df.head(1)

,name,total_money_raised,offices
0,Wetpaint,$39.8M,"[{'city': 'Seattle', 'latitude': 47.603122, 'l..."


In [36]:
# cities = []
# latitudes = []
# longitudes = []
# addresses = []
# money = []

# for index, row in df.iterrows():
    
#     try:
#         cities.append(row["offices"]["city"])
#         latitudes.append(row["offices"]["latitude"])
#         longitudes.append(row["offices"]["longitude"])
    
#     except IndexError:
#         cities.append(None)
#         latitudes.append(None)
#         longitudes.append(None)

    
# df["city"] = cities
# df["latitude"] = latitudes
# df["longitude"] = longitudes

In [37]:
# df.city.value_counts().head(5) 
# Instead of Palo Alto, London. Being in Europe makes it easier for the employees
# to network with successful companies and/or employees from different countries and cultures.
# The US is massively big, so traveling to other countries for work becomes difficult.

In [38]:
# df2 = df.copy()
# df2["total_money_raised"] = df2["total_money_raised"].replace('M|\$|€|C|£', "", regex = True).replace('B', "000", regex = True)
# df2["total_money_raised"] = pd.to_numeric(df2["total_money_raised"])

# df2["total_money_raised"].max()

# df2.to_csv("filtered_companies.csv", index=False)


In [137]:
df3_topcompanies = df2.loc[df2["city"].isin(['San Francisco', 'New York', 'London'])].sort_values("total_money_raised", ascending=False).groupby(["city"]).first().reset_index()
df3_topcompanies

,city,name,total_money_raised,latitude,longitude
0,London,OANDA,117.0,51.519204,-0.162610
1,New York,AOL,1000.0,40.731132,-73.991931
2,San Francisco,Xero,244.0,37.779507,-122.390710


In [233]:
def creating_airports_df(df, city):
    
    type_ = []

    new = df.loc[(airports["municipality"] == city) & (~airports["type"].isin(["closed", "seaplane_base", "heliport"]))]
    
    new = new.filter(['name','latitude_deg','longitude_deg', 'municipality'], axis=1).reset_index(drop=True)
    
    new.rename(columns = {'latitude_deg': 'lat', 'longitude_deg': 'lon', 'municipality':'city'}, inplace=True)
    
    for index, row in new.iterrows():
        type_.append({"typepoint": {"type": "Point", "coordinates": [row["lat"], row["lon"]]}})
        
    new['type'] = pd.Series(type_)
    new['address'] = None
    new['type_of_place'] = 'Airport'
    new['distance_from_company'] = 0

    return new



In [316]:
#airports = pd.read_csv("../Project_3_Company_Headquarters/dataframes/airports.csv")

airports_london = creating_airports_df(airports, "London")
airports_newyork = creating_airports_df(airports, "New York")
airports_sanfrancisco = creating_airports_df(airports, "San Francisco")

In [317]:
#airports_newyork = airports_newyork.drop(airports_newyork.index[0], axis=0)
airports_newyork

,name,lat,lon,city,type,address,type_of_place,distance_from_company
0,Whaley Farm Airstrip,53.067137,-0.148616,New York,"{'typepoint': {'type': 'Point', 'coordinates':...",None,Airport,0
1,John F Kennedy International Airport,40.639447,-73.779317,New York,"{'typepoint': {'type': 'Point', 'coordinates':...",None,Airport,0
2,La Guardia Airport,40.777199,-73.872597,New York,"{'typepoint': {'type': 'Point', 'coordinates':...",None,Airport,0


# Accessing the API

In [44]:
token_fsq = os.getenv("key")
token_fsq

'fsq3+bzNB46UzQcV4R4Uc5gDnPvzCPqR93jv1QubIBPd4oY='

In [361]:
def get_results_from_foursquare (query, location, limit):

    ll = f"{location[1]}%2C{location[0]}"
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={ll}&sort=DISTANCE&limit={str(limit)}"

    headers = {
        "accept": "application/json",
        "Authorization": token_fsq,
    }

    response = requests.get(url, headers=headers).json()
    
    return response

In [369]:
def making_requests(query):
    
    location = []
    dict_of_dfs = dict()

    for index, row in df3_topcompanies.iterrows():

        location.append(row["longitude"])
        location.append(row["latitude"])

        places = get_results_from_foursquare(query, location, 50)
        print(places)

        dict_of_dfs[row["city"]] = creating_dfs(places, query, row["city"])

        location = []
               
    return dict_of_dfs

In [368]:
def creating_dfs(res, type_of_place, city):
    
    new_list = []
    
    for i in res["results"]:
    
        name = i["name"]
        address =  i["location"]["formatted_address"]
        lat = i["geocodes"]["main"]["latitude"]
        lon = i["geocodes"]["main"]["longitude"]
        distance_from_company = i["distance"]

        type_ = {"typepoint": 
                              {"type": "Point", 
                               "coordinates": [lat, lon]}}

        new_list.append({"name":name, "lat":lat, "lon":lon, "city": city, "type":type_, "address": address, "type_of_place": type_of_place, "distance_from_company": distance_from_company})
        
    df = pd.DataFrame.from_records(new_list)
        
    return df


In [370]:
bars = making_requests("Bar")
nightclubs = making_requests("Night Club")
elementary_schools = making_requests("Elementary School")
nurseries = making_requests("Nursery School")


{'results': [{'fsq_id': '4ccc9130c9b8468833cdbdc3', 'categories': [{'id': 13009, 'name': 'Cocktail Bar', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/cocktails_', 'suffix': '.png'}}, {'id': 13013, 'name': 'Hotel Bar', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_bar_', 'suffix': '.png'}}, {'id': 13065, 'name': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}], 'chains': [], 'distance': 229, 'geocodes': {'main': {'latitude': 51.521536, 'longitude': -0.162908}, 'roof': {'latitude': 51.521536, 'longitude': -0.162908}}, 'link': '/v3/places/4ccc9130c9b8468833cdbdc3', 'location': {'address': '222 Marylebone Rd', 'admin_region': 'England', 'country': 'GB', 'formatted_address': '222 Marylebone Rd, London, Greater London, NW1 6JQ', 'locality': 'London', 'neighborhood': ['Regent s Park Camden Town'], 'post_town': 'London', 'postcode': 'NW1 6JQ', 'region': 'Greater London'}, 'name': 'The 

{'results': [{'fsq_id': '3fd66200f964a520cae91ee3', 'categories': [{'id': 13065, 'name': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}], 'chains': [], 'distance': 220, 'geocodes': {'main': {'latitude': 40.731887, 'longitude': -73.994365}, 'roof': {'latitude': 40.731887, 'longitude': -73.994365}}, 'link': '/v3/places/3fd66200f964a520cae91ee3', 'location': {'address': '33 University Pl', 'census_block': '360610059004000', 'country': 'US', 'cross_street': 'at E 9th St.', 'dma': 'New York', 'formatted_address': '33 University Pl (at E 9th St.), New York, NY 10003', 'locality': 'New York', 'neighborhood': ['Downtown'], 'postcode': '10003', 'region': 'NY'}, 'name': 'Knickerbocker Bar & Grill', 'related_places': {}, 'timezone': 'America/New_York'}, {'fsq_id': '4b01eca4f964a520984622e3', 'categories': [{'id': 13046, 'name': 'Ice Cream Parlor', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_', 'suffix': '.pn

{'results': [{'fsq_id': '59584985f4b5252a67a18dbe', 'categories': [{'id': 10000, 'name': 'Arts and Entertainment', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/default_', 'suffix': '.png'}}, {'id': 13013, 'name': 'Hotel Bar', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_bar_', 'suffix': '.png'}}, {'id': 13065, 'name': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}}], 'chains': [], 'distance': 39, 'geocodes': {'main': {'latitude': 37.77927, 'longitude': -122.390797}, 'roof': {'latitude': 37.77927, 'longitude': -122.390797}}, 'link': '/v3/places/59584985f4b5252a67a18dbe', 'location': {'address': '138 King St', 'census_block': '060750607011002', 'country': 'US', 'cross_street': 'btwn 2nd St & 3rd St', 'dma': 'San Francisco-Oakland-San Jose', 'formatted_address': '138 King St (btwn 2nd St & 3rd St), San Francisco, CA 94107', 'locality': 'San Francisco', 'neighborhood': ['

{'results': [{'fsq_id': '97b33b1e55a545061d100ae2', 'categories': [{'id': 10032, 'name': 'Night Club', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/nightclub_', 'suffix': '.png'}}], 'chains': [], 'distance': 280, 'geocodes': {'main': {'latitude': 51.519402, 'longitude': -0.158568}}, 'link': '/v3/places/97b33b1e55a545061d100ae2', 'location': {'address': '83-87 Crawford St', 'admin_region': 'England', 'country': 'GB', 'formatted_address': '83-87 Crawford St, London, W1H 2HB', 'locality': 'London', 'post_town': 'London', 'postcode': 'W1H 2HB'}, 'name': 'London Lux Guestlist', 'related_places': {}}, {'fsq_id': '4ed17ddfb634dd2992e0e052', 'categories': [{'id': 10032, 'name': 'Night Club', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/nightclub_', 'suffix': '.png'}}], 'chains': [], 'distance': 707, 'geocodes': {'main': {'latitude': 51.513201, 'longitude': -0.087121}}, 'link': '/v3/places/4ed17ddfb634dd2992e0e052', 'location': {'address': '24 Cor

{'results': [{'fsq_id': '3fd66200f964a52035e41ee3', 'categories': [{'id': 10039, 'name': 'Music Venue', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/musicvenue_', 'suffix': '.png'}}, {'id': 13016, 'name': 'Lounge', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/default_', 'suffix': '.png'}}, {'id': 13022, 'name': 'Sports Bar', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/sportsbar_', 'suffix': '.png'}}], 'chains': [], 'distance': 252, 'geocodes': {'main': {'latitude': 40.731707, 'longitude': -73.989031}, 'roof': {'latitude': 40.731707, 'longitude': -73.989031}}, 'link': '/v3/places/3fd66200f964a52035e41ee3', 'location': {'address': '125 E 11th St', 'census_block': '360610042002001', 'country': 'US', 'cross_street': 'btwn 3rd & 4th Ave', 'dma': 'New York', 'formatted_address': '125 E 11th St (btwn 3rd & 4th Ave), New York, NY 10003', 'locality': 'New York', 'neighborhood': ['Downtown'], 'postcode': '10003', '

...

{'results': [{'fsq_id': '566c60e4498efb2d84da6123', 'categories': [{'id': 12009, 'name': 'Education', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_', 'suffix': '.png'}}], 'chains': [], 'distance': 720, 'geocodes': {'main': {'latitude': 40.72489, 'longitude': -73.989811}, 'roof': {'latitude': 40.72489, 'longitude': -73.989811}}, 'link': '/v3/places/566c60e4498efb2d84da6123', 'location': {'address': '44 E 2nd St', 'census_block': '360610036021000', 'country': 'US', 'cross_street': '2nd Avenue', 'dma': 'New York', 'formatted_address': '44 E 2nd St (2nd Avenue), New York, NY 10003', 'locality': 'New York', 'neighborhood': ['Downtown'], 'postcode': '10003', 'region': 'NY'}, 'name': 'Nord Anglia International School', 'related_places': {}, 'timezone': 'America/New_York'}, {'fsq_id': '4b473bd9f964a5204b2d26e3', 'categories': [{'id': 12009, 'name': 'Education', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_', 'suffix': '.png'}}], 'chai

{'results': [{'fsq_id': '4af2f8fdf964a52079e921e3', 'categories': [{'id': 12058, 'name': 'Elementary School', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_', 'suffix': '.png'}}], 'chains': [], 'distance': 1937, 'geocodes': {'main': {'latitude': 37.763563, 'longitude': -122.399263}, 'roof': {'latitude': 37.763563, 'longitude': -122.399263}}, 'link': '/v3/places/4af2f8fdf964a52079e921e3', 'location': {'address': '1555 Mariposa St', 'census_block': '060750227041011', 'country': 'US', 'cross_street': '', 'dma': 'San Francisco-Oakland-San Jose', 'formatted_address': '1555 Mariposa St, San Francisco, CA 94107', 'locality': 'San Francisco', 'neighborhood': ['Potrero District'], 'postcode': '94107', 'region': 'CA'}, 'name': 'Live Oak School', 'related_places': {'children': [{'fsq_id': '5c0583a2b77c77002c51a8c8', 'name': 'Pachyderm.io HQ'}]}, 'timezone': 'America/Los_Angeles'}, {'fsq_id': '4ada1e70f964a520a01e21e3', 'categories': [{'id': 12058, 'name': 'Elementary 

{'results': [{'fsq_id': '595f55aa625a6632c60fe2b9', 'categories': [{'id': 12055, 'name': 'Nursery School', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_', 'suffix': '.png'}}], 'chains': [], 'distance': 655, 'geocodes': {'main': {'latitude': 51.518239, 'longitude': -0.172147}, 'roof': {'latitude': 51.518239, 'longitude': -0.172147}}, 'link': '/v3/places/595f55aa625a6632c60fe2b9', 'location': {'address': '11a, West End Quay', 'address_extended': 'Unit 11a', 'admin_region': 'England', 'country': 'GB', 'cross_street': '', 'formatted_address': '11a, West End Quay, London, Greater London, W2 1JB', 'locality': 'London', 'neighborhood': ['Paddington'], 'post_town': 'London', 'postcode': 'W2 1JB', 'region': 'Greater London'}, 'name': 'Hope Montessori Nursery School', 'related_places': {}, 'timezone': 'Europe/London'}, {'fsq_id': '556c1fe5498e618f83fe321a', 'categories': [{'id': 12055, 'name': 'Nursery School', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories

{'results': [{'fsq_id': '5a6f39f12c7eb951c906fec0', 'categories': [{'id': 11026, 'name': 'Daycare', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/daycare_', 'suffix': '.png'}}], 'chains': [], 'distance': 1547, 'geocodes': {'main': {'latitude': 40.718759, 'longitude': -73.983154}, 'roof': {'latitude': 40.718759, 'longitude': -73.983154}}, 'link': '/v3/places/5a6f39f12c7eb951c906fec0', 'location': {'address': '112 Ridge St', 'census_block': '360610022012000', 'country': 'US', 'cross_street': '', 'dma': 'New York', 'formatted_address': '112 Ridge St, New York, NY 10002', 'locality': 'New York', 'neighborhood': ['Downtown'], 'postcode': '10002', 'region': 'NY'}, 'name': 'All My Children Day Care & Nursery Schools', 'related_places': {}, 'timezone': 'America/New_York'}, {'fsq_id': '50d4d5eee4b0f43837b23604', 'categories': [{'id': 11026, 'name': 'Daycare', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/daycare_', 'suffix': '.png'}}], 'chains': [], 'distan

{'results': [{'fsq_id': '5120081ce4b0d10dc2005ab1', 'categories': [{'id': 12056, 'name': 'Preschool', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_', 'suffix': '.png'}}], 'chains': [], 'distance': 1876, 'geocodes': {'main': {'latitude': 37.776862, 'longitude': -122.411659}, 'roof': {'latitude': 37.776862, 'longitude': -122.411659}}, 'link': '/v3/places/5120081ce4b0d10dc2005ab1', 'location': {'address': '685 Natoma St', 'census_block': '060750176032002', 'country': 'US', 'cross_street': '8th', 'dma': 'San Francisco-Oakland-San Jose', 'formatted_address': '685 Natoma St (8th), San Francisco, CA 94103', 'locality': 'San Francisco', 'neighborhood': ['Civic Center'], 'postcode': '94103', 'region': 'CA'}, 'name': 'Judith Baker Child Development Center', 'related_places': {}, 'timezone': 'America/Los_Angeles'}, {'fsq_id': '4af0759df964a5204bdc21e3', 'categories': [{'id': 11026, 'name': 'Daycare', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/da

In [371]:
bars1 = bars
nightclubs1 = nightclubs
elementary_schools1 = elementary_schools
nurseries1 = nurseries


In [375]:
bars["San Francisco"]

,name,lat,lon,city,type,address,type_of_place,distance_from_company
0,Bar VIA,37.779270,-122.390797,San Francisco,"{'typepoint': {'type': 'Point', 'coordinates':...","138 King St (btwn 2nd St & 3rd St), San Franci...",Bar,39
1,Bravado Italian Coffee Bar and Lounge,37.778826,-122.391488,San Francisco,"{'typepoint': {'type': 'Point', 'coordinates':...","170 King St (at 3rd St.), San Francisco, CA 94107",Bar,98
2,California Wine Bar,37.778625,-122.390257,San Francisco,"{'typepoint': {'type': 'Point', 'coordinates':...","101 King St, San Francisco, CA 94107",Bar,108
3,Field Club Bar,37.778620,-122.389899,San Francisco,"{'typepoint': {'type': 'Point', 'coordinates':...","24 Willie Mays Plz, San Francisco, CA 94107",Bar,109
4,The Bar San Francisco,37.778880,-122.392495,San Francisco,"{'typepoint': {'type': 'Point', 'coordinates':...","701 3rd St (Townsend), San Francisco, CA 94107",Bar,167
5,Kaiyo Rooftop,37.778736,-122.392370,San Francisco,"{'typepoint': {'type': 'Point', 'coordinates':...","701 3rd St, San Francisco, CA 94107",Bar,169
6,Cable Car Bar,37.778403,-122.389312,San Francisco,"{'typepoint': {'type': 'Point', 'coordinates':...","24 Willie Mays Plz (at AT&T Park), San Francis...",Bar,181
7,Alchemist Bar & Lounge,37.779264,-122.393019,San Francisco,"{'typepoint': {'type': 'Point', 'coordinates':...","679 3rd St (Townsend St), San Francisco, CA 94107",Bar,207
8,The Ketel One Bar,37.778338,-122.388448,San Francisco,"{'typepoint': {'type': 'Point', 'coordinates':...","24 Willie Mays Plz, San Francisco, CA 94107",Bar,240
9,baR/GA,37.781649,-122.393446,San Francisco,"{'typepoint': {'type': 'Point', 'coordinates':...","35 S Park St, San Francisco, CA 94107",Bar,350


In [376]:
df3_topcompanies

,city,name,total_money_raised,latitude,longitude,distance_from_company
0,London,OANDA,117.0,51.519204,-0.162610,6496.379543
1,New York,AOL,1000.0,40.731132,-73.991931,5860.177410
2,San Francisco,Xero,244.0,37.779507,-122.390710,5864.933732


In [356]:
# def distances(df_topcompanies, dict_places):
    
#     distance_from_company = []
    
#     for index, row in df_topcompanies.iterrows():
#         company_loc = (row["latitude"], row["longitude"])
        
#         for place in dict_places.values():
            
#             distance_from_company = []
            
#             for index, row in place.iterrows():
            
#                 location = (row["lat"], row["lon"])

#                 distance_from_company.append(hs.haversine(location, company_loc))

                
#         place["distance_from_company"] = pd.Series(distance_from_company)
        
#         company_loc = list(company_loc)
#         company_loc = []
#         company_loc = [] = tuple(company_loc)
        
#         location = list(location)
#         location = []
#         location = [] = tuple(location)


In [377]:
def distances_airports(coord_company, df):
    
    distance_from_company = []
    
    for i in range(df.shape[0]):    
        
        distance_from_company.append(hs.haversine((df["lat"].iloc[i], df["lon"].iloc[i]), (coord_company)))
#         print((df["lat"].iloc[i], df["lon"].iloc[i]))
#         print(distance_from_company)

    df["distance_from_company"] = pd.Series(distance_from_company)

    

In [258]:
hs.haversine((51.519204, -0.162610), (37.023019, -84.12225))

6496.379570093406

In [378]:
air = {"London" : airports_london,
      "New York": airports_newyork,
      "San Francisco": airports_sanfrancisco}

airports1 = air

# distances(df3_topcompanies, nurseries)
# distances(df3_topcompanies, bars)
# distances(df3_topcompanies, nightclubs)
# distances(df3_topcompanies, elementary_schools)
# distances(df3_topcompanies, air)

distances_airports((df3_topcompanies["latitude"].iloc[1], df3_topcompanies["longitude"].iloc[1]), airports_newyork)
distances_airports((df3_topcompanies["latitude"].iloc[2], df3_topcompanies["longitude"].iloc[2]), airports_sanfrancisco)
distances_airports((df3_topcompanies["latitude"].iloc[0], df3_topcompanies["longitude"].iloc[0]), airports_london)



In [379]:

def concat_dfs(list_, city):
    
    df1 = pd.DataFrame()
    
    for dic in list_:

        for df in dic.values():                

            if df["city"].iloc[0] == city:
                df1 = pd.concat([df1, df], axis=0).reset_index(drop = True)

    return df1



In [380]:
london = concat_dfs([bars, nightclubs, nurseries, elementary_schools, air], "London")
new_york = concat_dfs([bars, nightclubs, nurseries, elementary_schools, air], "New York")
san_francisco = concat_dfs([bars, nightclubs, nurseries, elementary_schools, air], "San Francisco")

In [383]:
london.to_csv("london_places.csv", index=False)
new_york.to_csv("ny_places.csv", index=False)
san_francisco.to_csv("sanfrancisco_places.csv", index=False)


In [384]:
def city_weights(df):
    
    city = df["city"].iloc[0] 
    weights = {"bars" : round((df.loc[df["type_of_place"] == "Bar"]["distance_from_company"].head(5).mean())*0.6, 2),
                "nightclubs" : round((df.loc[df["type_of_place"] == "Night Club"]["distance_from_company"].head(5).mean())*0.6, 2),
                "elementary_schools": round(df.loc[df["type_of_place"] == "Elementary School"]["distance_from_company"].min()*0.8, 2),
                "nurseries": round(df.loc[df["type_of_place"] == "Nursery School"]["distance_from_company"].min()*0.8, 1),
                "airports": round(df.loc[df["type_of_place"] == "Airport"]["distance_from_company"].min(), 1),
                'city' : city
}
    return weights

    

In [385]:
london_weight = city_weights(london)
london_weight

{'bars': 187.08,
 'nightclubs': 623.64,
 'elementary_schools': 1326.4,
 'nurseries': 524.0,
 'airports': 15.2,
 'city': 'London'}

In [386]:
newyork_weight = city_weights(new_york)
sanfrancisco_weight = city_weights(san_francisco)

In [387]:
sanfrancisco_weight

{'bars': 62.52,
 'nightclubs': 526.44,
 'elementary_schools': 1549.6,
 'nurseries': 1500.8,
 'airports': 17.9,
 'city': 'San Francisco'}

In [388]:
newyork_weight

{'bars': 192.84,
 'nightclubs': 417.6,
 'elementary_schools': 576.0,
 'nurseries': 1237.6,
 'airports': 11.3,
 'city': 'New York'}

In [399]:
def city_score(city_weight1, city_weight2, city_weight3):
    
    keys_from_cityweight = list(city_weight1.keys())
    score = []
    
    for i in range(len(city_weight1)):
            
        if (city_weight1[keys_from_cityweight[i]] < city_weight2[keys_from_cityweight[i]]) & (city_weight1[keys_from_cityweight[i]] < city_weight3[keys_from_cityweight[i]]):
            score.append(city_weight1['city'])
            
        elif (city_weight2[keys_from_cityweight[i]] < city_weight1[keys_from_cityweight[i]]) & (city_weight2[keys_from_cityweight[i]] < city_weight3[keys_from_cityweight[i]]):
            score.append(city_weight2['city'])
        
        elif (city_weight3[keys_from_cityweight[i]] < city_weight2[keys_from_cityweight[i]]) & (city_weight3[keys_from_cityweight[i]] < city_weight1[keys_from_cityweight[i]]):
            score.append(city_weight3['city'])
    
    return f"The best city for the HQ is {s.multimode(score)}"
    

In [400]:
city_score(london_weight, newyork_weight, sanfrancisco_weight)

"The best city for the HQ is ['New York']"

# Maps

In [401]:
map_london = Map(location=[df3_topcompanies["latitude"].iloc[0], df3_topcompanies["longitude"].iloc[0]], zoom_start=15)
map_newyork = Map(location=[df3_topcompanies["latitude"].iloc[1], df3_topcompanies["longitude"].iloc[1]], zoom_start=15)
map_sanfrancisco = Map(location=[df3_topcompanies["latitude"].iloc[2], df3_topcompanies["longitude"].iloc[2]], zoom_start=15)

In [402]:
map_london

In [420]:
def creating_maps(df_company, df, map_):
    
    headquarters = {"location": [df_company["latitude"], df_company["longitude"]], "tooltip": df_company["name"]}
    icon = Icon (
                color="blue",
                opacity = 0.6,
                prefix = "fa",
                icon="building-o",
                icon_color = "black"
            )
    new_marker = Marker(**headquarters, icon = icon, radius = 2)
    new_marker.add_to(map_)
    
    
    for index, row in df.iterrows():

        #1. MARKER without icon
        place = {"location": [row["lat"], row["lon"]], "tooltip": row["distance_from_company"]}

        #2. Icon
        if row["type_of_place"] == "Night Club":        
            icon = Icon (
                color="red",
                opacity = 0.6,
                prefix = "fa",
                icon="glass",
                icon_color = "white"
            )
            
        elif row["type_of_place"] == "Bar":
            icon = Icon (
                color="orange",
                opacity = 0.6,
                prefix = "fa",
                icon="beer",
                icon_color = "white"
            )
            
        elif row["type_of_place"] == "Nursery School":
            icon = Icon (
                color="white",
                opacity = 0.6,
                prefix = "fa",
                icon="minus-circle",
                icon_color = "black"
           )
        elif row["type_of_place"] == "Elementary School":
            icon = Icon (
                color="black",
                opacity = 0.6,
                prefix = "fa",
                icon="plus-circle",
                icon_color = "white"
            )
        else:
            icon = Icon (
                color="blue",
                opacity = 0.6,
                prefix = "fa",
                icon="plane",
                icon_color = "white",
            )
        #3. Marker
        new_marker = Marker(**place, icon = icon, radius = 2)

        #4. Add the Marker
        new_marker.add_to(map_)

In [421]:
creating_maps(df3_topcompanies.loc[df3_topcompanies['city'] == 'London'], london, map_london)
map_london

In [422]:
creating_maps(df3_topcompanies.loc[df3_topcompanies['city'] == 'New York'], new_york, map_newyork)
map_newyork

In [423]:
creating_maps(df3_topcompanies.loc[df3_topcompanies['city'] == 'San Francisco'], san_francisco, map_sanfrancisco)
map_sanfrancisco